<a href="https://colab.research.google.com/github/AdityaJ9082/NLP/blob/main/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Query,Key,Value

In [ ]:
import numpy as np
import math

In [ ]:
l,d_k,d_v=4,8,8#seq length,query and key vector dim
q=np.random.randn(l,d_k)
k=np.random.randn(l,d_k)
v=np.random.randn(l,d_v)


In [ ]:
print("Q",q)
print("K",k)
print("V",v)

Q [[ 1.78805178 -0.97709767 -0.56062213  0.14574544 -0.46574268  0.07245262
   0.28151529 -0.65062424]
 [ 0.25239774  0.61786864  0.00446839 -1.18138859  1.61240732 -0.42076995
  -0.66835692  1.01128964]
 [-0.10833257 -0.68804893 -0.98784707 -0.4058536   1.618865    0.87893645
  -1.34027978  0.04356557]
 [-0.96836213 -0.19840949  0.21115702 -1.38028163  0.31577939  0.8340221
   0.33326597 -0.07538146]]
K [[-0.61201758  2.23609807 -0.85985637  0.2453094  -0.59652938 -0.29199138
  -1.50659413  0.38821032]
 [-0.94631208  1.15549986 -0.25802011 -2.24477849  0.16352622 -0.09712039
   0.30080652 -0.02394309]
 [-0.63830042 -0.36017393  0.02671005 -0.2671434   0.4601723  -0.48624236
  -1.01478747 -1.04662445]
 [-0.47462319 -0.75215272 -0.1165112   0.19233326  0.9004559  -1.44158313
  -1.61739572 -0.09663339]]
V [[-0.74377148 -1.90359234 -1.33417314  1.1418333   1.03203071 -1.86270414
   0.52654133  0.96171157]
 [ 0.12167117 -1.12273009 -0.69987221 -0.08198949 -0.74948342  0.31007397
   0.63690

#Self Attention

self attention=softmax((q*k.t/(d_k)^1/2)+M)*V

In [ ]:
np.matmul(q,k.T)

array([[-3.1814328 , -2.98654384, -0.69756864, -0.93665055],
       [ 1.49404368,  3.20517955,  0.49845487,  2.22948349],
       [ 0.09143132,  0.24856806,  2.03108471,  2.9601806 ],
       [-1.33442856,  3.80374551,  0.54441902, -1.13094023]])

In [ ]:
q.var(),k.var(),np.matmul(q,k.T).var()#the variance of multiplication is much higher

(0.6850279138706385, 0.7221737108881208, 4.124295087233898)

In [ ]:
scaled=np.matmul(q,k.T)/np.sqrt(d_k)
print("Scaled",scaled)
print("Scaled Variance",scaled.var())#low var and almost have same range

Scaled [[-1.12480636 -1.0559027  -0.24662776 -0.33115598]
 [ 0.52822421  1.1332021   0.17623041  0.78824145]
 [ 0.03232585  0.08788208  0.71809688  1.04658189]
 [-0.47179174  1.34482712  0.19248119 -0.39984775]]
Scaled Variance 0.5155368859042373


#Masking


*   Masking is required to ensure that words dont get context from words generated in the future
*  Only required in decoders and not required in encoders





In [ ]:
mask=np.tril(np.ones((l,l)))

In [ ]:
mask#my can only look at iself and nothing else. name can only look at my and name and nothing else

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [ ]:
mask[mask==0]=-np.infty
mask[mask==1]=0#we do this because we are adding it to scaled and also we are using softmax function

In [ ]:
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [ ]:
scaled+mask

array([[-1.12480636,        -inf,        -inf,        -inf],
       [ 0.52822421,  1.1332021 ,        -inf,        -inf],
       [ 0.03232585,  0.08788208,  0.71809688,        -inf],
       [-0.47179174,  1.34482712,  0.19248119, -0.39984775]])

#Softmax

The values would all be added up to one and also be interpretible and stable and

In [ ]:
def softmax(x):
  return (np.exp(x).T/np.sum(np.exp(x),axis=-1)).T

In [ ]:
attention=softmax(scaled+mask)

In [ ]:
attention#my will only focis on my. name will only focus on my and name

array([[1.        , 0.        , 0.        , 0.        ],
       [0.35320566, 0.64679434, 0.        , 0.        ],
       [0.24737593, 0.26150814, 0.49111593, 0.        ],
       [0.098341  , 0.60489806, 0.19108417, 0.10567677]])

In [ ]:
new_v=np.matmul(attention,v)

In [ ]:
new_v#capture the better context of words

array([[-0.74377148, -1.90359234, -1.33417314,  1.1418333 ,  1.03203071,
        -1.86270414,  0.52654133,  0.96171157],
       [-0.18400808, -1.39853506, -0.92391089,  0.35027164, -0.12024255,
        -0.45736355,  0.5979217 , -0.33420859],
       [-0.54531394, -1.37833243, -0.37719803,  1.02697089,  0.30913681,
        -0.35006538, -0.08939905,  0.89764945],
       [-0.06276994, -1.03114936, -0.45291033,  0.41747771, -0.12488165,
         0.19042569,  0.0936519 , -0.16141823]])

In [ ]:
v#the first vector of old and new vector that is vector for my word will be same

array([[-0.74377148, -1.90359234, -1.33417314,  1.1418333 ,  1.03203071,
        -1.86270414,  0.52654133,  0.96171157],
       [ 0.12167117, -1.12273009, -0.69987221, -0.08198949, -0.74948342,
         0.31007397,  0.63690156, -1.04189309],
       [-0.80050503, -1.2498606 ,  0.27664867,  1.55961085,  0.50870527,
         0.06034406, -0.78638794,  1.89814385],
       [ 0.8491789 ,  0.70041556,  0.46161283,  0.53717597,  1.22811247,
         1.65137481, -1.82748711,  0.10920047]])

Putting everything in one function

In [ ]:
def softmax(x):
  return (np.exp(x).T/np.sum(np.exp(x),axis=-1)).T

def scaled_attention(q,k,v,mask=None):
  d_k=q.shape[-1]
  scaled=np.matmul(q,k.T)/math.sqrt(d_k)
  if mask is not None:
    scaled=scaled+mask
  attention=softmax(scaled)
  output=np.matmul(attention,v)
  return output,attention

In [ ]:
values,attention=scaled_attention(q,k,v,mask=None)#mask is none for encoder and not none for decoder
print("Q\n",q)
print("K\n",k)
print("Old V\n",v)
print("New V\n",values)
print("Attention\n",attention)

Q
 [[ 1.78805178 -0.97709767 -0.56062213  0.14574544 -0.46574268  0.07245262
   0.28151529 -0.65062424]
 [ 0.25239774  0.61786864  0.00446839 -1.18138859  1.61240732 -0.42076995
  -0.66835692  1.01128964]
 [-0.10833257 -0.68804893 -0.98784707 -0.4058536   1.618865    0.87893645
  -1.34027978  0.04356557]
 [-0.96836213 -0.19840949  0.21115702 -1.38028163  0.31577939  0.8340221
   0.33326597 -0.07538146]]
K
 [[-0.61201758  2.23609807 -0.85985637  0.2453094  -0.59652938 -0.29199138
  -1.50659413  0.38821032]
 [-0.94631208  1.15549986 -0.25802011 -2.24477849  0.16352622 -0.09712039
   0.30080652 -0.02394309]
 [-0.63830042 -0.36017393  0.02671005 -0.2671434   0.4601723  -0.48624236
  -1.01478747 -1.04662445]
 [-0.47462319 -0.75215272 -0.1165112   0.19233326  0.9004559  -1.44158313
  -1.61739572 -0.09663339]]
Old V
 [[-0.74377148 -1.90359234 -1.33417314  1.1418333   1.03203071 -1.86270414
   0.52654133  0.96171157]
 [ 0.12167117 -1.12273009 -0.69987221 -0.08198949 -0.74948342  0.31007397
   

In [ ]:
values

array([[-0.09895298, -0.68247475, -0.05940459,  0.89623108,  0.62326473,
         0.33884668, -0.70634923,  0.69587191],
       [ 0.00360059, -0.81345183, -0.37722324,  0.5764803 ,  0.33326151,
         0.184065  , -0.25465972,  0.1097699 ],
       [ 0.02015619, -0.53539523, -0.03705825,  0.8283579 ,  0.6817836 ,
         0.46152333, -0.79419786,  0.57793153],
       [-0.06276994, -1.03114936, -0.45291033,  0.41747771, -0.12488165,
         0.19042569,  0.0936519 , -0.16141823]])

In [ ]:
l=[1,2,3]
print(*l)

1 2 3


#Encoder

In [ ]:
import torch
from torch import nn
import math
import torch.nn.functional as F

In [ ]:
# import numpy as np
# x=np.zeros(shape=(30,200,512))


In [ ]:
def scaled_attention_prod(q,k,v,mask=None):
  #q,k,v=30,8,200,64
  d_k=q.size()[-1]#64
  scaled=F.matmul(q,k.transpose(-1,-2))/np.sqrt(d_k)#k_size=batch_size,msl,emb after transpose->batch_size,emb,msl---->30,8,200,200
  if mask is not None:
    scaled=scaled+mask#broadcasting since shape of scaled is 30,8,200,200 and mask is 200,200
  attention=F.softmax(scaled,dim=-1)# 30,8,200,200 -> also adding only values so dim=-1
  output=F.matmul(attention,v)#30,8,200,64 -> 64 dimensional embedding
  return attention,output

In [ ]:
a=[-(i+1) for i in range(len([512]))]
a

[-1]

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self,d_model,num_heads):
    super().__init__()
    self.d_model=d_model#512
    self.num_heads=num_heads#8
    self.head_dim=d_model//num_heads#64
    self.qkv_layer=nn.Linear(d_model,d_model*3)#they are not individual q,kv but they are stacked tensors 512*1536
    self.linear_layer=nn.Linear(d_model,d_model)#512*512

  def forward(self,x):
    batch_size,m_s_l,d_model=x.size()#30,200,512
    qkv=self.qkv_layer(x)#30,200,1536
    qkv=qkv.reshape(batch_size,m_s_l,self.num_heads,3*self.head_dim)#30,200,8,192 ->dividing the 1536 into 8 heads each having 64 dims for q,k,v respectively
    qkv=qkv.permute(0,2,1,3)#30,8,200,192
    q,k,v=qkv.chunk(3,dim=-1)#each 30,8,200,64
    values,attention=scaled_attention_prod(q,k,v,mask=None)#attention 30,8,200,200  value 30,8,200,64
    values=values.reshape(batch_size,m_s_l,self.head*self.head_dim)#30,200,512
    out=self.linear_layer(values)
    return out

class LayerNormalisation(nn.Module):
  def __init__(self,parameters_shape,eps=1e-5):
    super().__init__()
    self.parameters_shape=parameters_shape#[512]
    self.eps=eps
    self.gamma=nn.Parameter(torch.ones(parameters_shape))#learnable parameter [512] -> 512 dimesionals tensors
    self.beta=nn.Parameter(torch.ones(parameters_shape))

  def forward(self,inputs):#30,200,512
    dims=[-(i+1) for i in range(len(self.parameters_shape))]#[-1]
    mean=inputs.mean(dim=dims,keepdim=True)# 30,200,1 taking mean along last dimension that is embedding dimension for eg we a embedding of a word we take the mean of that whole embedding vector
    var=((inputs-mean)**2).mean(dim=dims,keepdim=True)#30,200,1
    std=(var+self.eps).sqrt()#30,200,1
    y=(inputs-mean)/std# 30,200,512 for every number in last dimension we subtract by mean and divide by std
    out=self.gamma*y+self.beta#30,200,512
    return out


class PositionWiseFeedForward(nn.Module):
  def __init__(self,d_model,ffn_hidden,drop_prob):
    super(PositionWiseFeedForward,self).__init__()
    self.linear1=nn.Linear(d_model,ffn_hidden)#512,2048
    self.linear2=nn.Linear(ffn_hidden,d_model)#2048,512
    self.relu=nn.ReLu()
    self.dropout=nn.Dropout(p=drop_prob)

  def forward(self,x):#30,200,512
    x=self.linear1(x)#30,200,2048
    x=self.relu(x)#30,200,512
    x=self.dropout(x)#30,200,512
    x=self.linear2(x)#30,200,512
    return x









In [ ]:
d_model=512
n_heads=8
drop_prob=0.1
batch_size=30
m_s_l=200
ffn_hidden=2048
num_layers=5

class Encoder(nn.Module):
  def __init__(self,d_model,ffn_hidden,num_heads,drop_prob,num_layers):
    super().__init__()
    self.layers=nn.Sequential(*[EncoderLayer(d_model,ffn_hidden,num_heads,drop_prob) for _ in range(num_layers)])

  def forward(self,x):
    x=self.layers(x)
    return x

In [ ]:
class EncoderLayer(nn.Module):
  def __init__(self,d_model,ffn_hidden,num_heads,drop_prob):
    super(EncoderLayer,self).__init__()
    self.attention=MultiHeadAttention(d_model=d_model,num_heads=num_heads)
    self.norm1=LayerNormalisation(parameters_shape=[d_model])
    self.dropout1=nn.Dropout(p=drop_prob)
    self.ffn=PositionWiseFeedForward(d_model=d_model,hidden=ffn_hidden,drop_prob=drop_prob)
    self.norm2=LayerNormalisation(parameters_shape=[d_model])
    self.dropout2=nn.Dropout(p=drop_prob)

  def forward(self,x):
    residual_conn=x#30,200,512
    x=self.attention(x,mask=None)#30,200,512
    x=self.dropout1(x)#30,200,512
    x=self.norm1(residual_conn+x)#30,200,512
    residual_x=x#30,200,512
    x=self.ffn(x)#30,200,512
    x=self.dropout2(x)#30,200,512
    x=self.norm2(x+residual_conn)#30,200,512
    return x#much more context aware


#Decoder

In [2]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
def scaled_dot_product(q,k,v,mask=None):
  #q=30,8,200,64 mask=200,200
  d_k=q.size()[-1]#64
  scaled=F.matmul(q,k.transpose(-1,-2))/np.sqrt(d_k)#k_size=batch_size,msl,emb after transpose->batch_size,emb,msl---->30,8,200,200
  print('scaled.size()',scaled.size())
  if mask is not None:
    print('mask size',mask.size())
    scaled=scaled+mask#broadcasting since shape of scaled is 30,8,200,200 and mask is 200,200
  attention=F.softmax(scaled,dim=-1)# 30,8,200,200 -> also adding only values so dim=-1  how much atention one word pays to every other words
  output=F.matmul(attention,v)#30,8,200,64 -> 64 dimensional embedding   context aware embeddings
  return attention,output



class MultiHeadAttention(nn.Module):
  def __init__(self,d_model,num_heads):
    super().__init__()
    self.d_model=d_model
    self.num_heads=num_heads
    self.head_dim=d_model//num_heads
    self.qkv_layer=nn.Linear(d_model,d_model*3)
    self.linear_layer=nn.Linear(d_model,d_model)

  def forward(self,x,mask=None):
    batch_size,m_s_l,d_model=x.size()#30,200,512
    print("x.size()",x.size())
    qkv=self.qkv_layer(x)#30,200,1536
    print('qkv.size',qkv.size())
    qkv=qkv.reshape(batch_size,m_s_l,self.num_heads,self.head_dim*3)#30,200,8,192
    print("qkv after reshape",qkv.size())
    qkv=qkv.permute(0,2,1,3)#30,8,200,192
    print("qkv after permutation",qkv.size())
    q,k,v=qkv.chunk(3,dim=-1)# each 30,8,200,64
    print("q size",q.size(),"k size",k.size(),"v size",v.size())
    values,attention=scaled_dot_product(q,k,v,mask)#values=30,8,200,64
    print('values.size()' ,values.size(),'attention.size()',attention.size())
    values=values.reshape(batch_size,m_s_l,self.num_heads*self.head_dim)#30,200,512
    print('values after reshaping',values.size())
    out=self.linear_layer(values)
    print('output after passing thorugh linear layer',out.size())
    return out

class LayerNormalisation(nn.Module):
  def __init__(self,parameters_shape,eps=1e-5):
    super().__init__()
    self.parameters_shape=parameters_shape#[512]
    self.eps=eps
    self.gamma=nn.Parameter(torch.ones(parameters_shape))#learnable parameter [512] -> 512 dimesionals tensors
    self.beta=nn.Parameter(torch.ones(parameters_shape))#512

  def forward(self,inputs):#30,200,512
    dims=[-(i+1) for i in range(len(self.parameters_shape))]#[-1]
    #if keepdim=False output of mean is #30,200 ,if keepdim=True op is #20,200,1
    mean=inputs.mean(dim=dims,keepdim=True) # 30,200,1 taking mean along last dimension that is embedding dimension for eg we a embedding of a word we take the mean of that whole embedding vector
    var=((inputs-mean)**2).mean(dim=dims,keepdim=True)#30,200,1
    std=(var+self.eps).sqrt()#30,200,1
    y=(inputs-mean)/std# 30,200,512 for every number in last dimension we subtract by mean and divide by std
    out=self.gamma*y+self.beta#30,200,512
    return out

class MultiHeadCrossAttention(nn.Module):
  def __init__(self,d_model,num_heads):
    super().__init__()
    self.d_model=d_model
    self.num_heads=num_heads
    self.head_dim=d_model//num_heads
    self.kv_layer=nn.Linear(d_model,d_model*2)
    self.q_layer=nn.Linear(d_model,d_model)
    self.linear_layer=nn.Linear(d_model,d_model)

  def forward(self,x,y,mask=None):
    batch_size,m_s_l,d_model=x.size()
    q=self.q_layer(y)#30,200,512
    print('q.size()',q.size())
    q=q.reshape(batch_size,m_s_l,self.num_heads,self.head_dims)#30,200,8,64
    kv=self.kv_layer(x)#30,200,1024
    print('kv.size',kv.size())
    kv=kv.reshape(batch_size,m_s_l,self.num_heads,2*self.head_dims)#30,200,8,128
    kv=kv.permute(0,2,1,3)#30,8,200,64
    q=q.permute(0,2,1,3)#30,8,200,64

    k,v=kv.chunk(2,dim=-1)#each 30,8,200,64
    values,attention=scaled_dot_product(q,k,v,mask)
    print('values.size',values.size,'attention.size()',attention.size())
    values=values.reshape(batch_size,m_s_l,self.num_heads*self.head_dim)
    print('value after reshaping',values.size())
    out=self.linear_layer(values)#30,200,512
    print('op after passing through linear layer',out.size())
    return out

class FeedForwardNetwork(nn.Module):
  def __init__(self,d_model,ffn_hidden,drop_prob):
    super(PositionWiseFeedForward,self).__init__()
    self.linear1=nn.Linear(d_model,ffn_hidden)#512,2048
    self.linear2=nn.Linear(ffn_hidden,d_model)#2048,512
    self.relu=nn.ReLu()
    self.dropout=nn.Dropout(p=drop_prob)

  def forward(self,x):#30,200,512
    x=self.linear1(x)#30,200,2048
    print('shape after 1st linear layer',x.size())
    x=self.relu(x)#30,200,2048
    print('shape after relu',x.size())
    x=self.dropout(x)#30,200,2048
    print('shape after dropout',x.size())
    x=self.linear2(x)#30,200,512
    print('shape after 2nd linear layer',x.size())
    return x



class DecoderLayer(nn.Module):
  def __init__(self,d_model,ffn_hidden,num_heads,drop_prob):
    super(DecoderLayer).__init__()
    self.self_attention=MultiHeadAttention(d_model=d_model,num_heads=num_heads)
    self.layernorm1=LayerNormalisation(parameters_shape=[d_model])
    self.dropout1=nn.Dropout(p=drop_prob)
    self.multiheadcrossattention=MultiHeadCrossAttention(d_model=d_model,num_heads=num_heads)
    self.layernorm2=LayerNormalisation(parameters_shape=[d_model])
    self.dropout2=nn.Dropout(p=drop_prob)
    self.feedforward=FeedForwardNetwork(d_model=d_model,ffn_hidden=ffn_hidden,drop_prob=drop_prob)
    self.layernorm3=LayerNormalisation(parameters_shape=[d_model])
    self.dropout3=nn.Dropout(p=drop_prob)

  def forward(self,x,y,decoder_mask):
    residual_conn=y
    print('Maked self attention')
    y=self.self_attention(y,mask=decoder_mask)#30,200,512
    print('Dropout')
    y=self.dropout1(y)#30,200,512
    print('Layer normalisation')
    y=self.layernorm1(y+residual_conn)#30,200,512
    residual_conn=y
    print('Cross Attention')
    y=self.multiheadcrossattention(x,y,mask=None)#30,200,512
    print('Dropout')
    y=self.dropout2(y)#30,200,512
    print('Layer normalisation')
    y=self.layernorm2(y+residual_conn)#30,200,512
    residual_conn=y#30,200,512
    print('Feed Forward')
    y=self.feedforward(y)


class SequentialDecoder(nn.Sequential):
  def forward(self,*inputs):
    x,y,mask=inputs
    for module in self._modules.values():
      y=module(x,y,mask)#produces output of one decoder layer after another
    return y

class Decoder(nn.Module):
  def __init__(self,d_model,ffn_hidden,num_heads,drop_prob,num_layers=1):
    super().__init__():
    self.layers=SequentialDecoder(*[DecoderLayer(d_model,ffn_hidden,num_heads,d_prob)for _ in range(num_layers)])

  def forward(self,x,y,mask):#if only use sequential then we cannot pass more than one value
    #x=30,200,512 y=#30,200,512 #mask=200,200
    y=self.layers(x,y,mask)
    return y



In [ ]:
d_model=512
d_prob=0.1
m_s_l=200
ffn_hidden=2048
batch_size=30
num_layers=5
num_heads=8

# class Decoder(nn.Module):
#   def __init__():

x=torch.randn((batch_size,m_s_l,d_model))#english sentence position encoded
y=torch.randn((batch_size,m_s_l,d_model))#kannada sentence position encoded
mask=torch.full([m_s_l,m_s_l],float('-inf'))
mask=torch.triu(mask,diagonal=1)
decoder=Decoder(d_model,ffn_hidden,num_heads,drop_prob,num_layers)
out=decoder(x,y,mask)


